# Somatic Mutation Identification with DNN-Boost

In this notebook, we will demonstrate the somatic mutation identification of variants dataset from tumor-normal paired pancreatic cancer data. We will first use feature selection on the data using XGBoost and then train a deep neural netwook on the data with the selected features according to XGBoost (using Tensorflow).

Note: Before running this tutorial, please install the scipy, pandas, matplotlib, Tensorflow, Sklearn, imblearn, and xgboost packages.

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import time

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay, average_precision_score, precision_recall_curve, plot_precision_recall_curve, plot_roc_curve
from sklearn.feature_selection import SelectFromModel

from sklearn import metrics
from sklearn.datasets import make_circles

from math import sqrt

from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Dense,Dropout,Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from sklearn.manifold import TSNE
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import EditedNearestNeighbours 
from imblearn.over_sampling import SMOTE

import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

# Import dataset of variants from tumor-normal pancreatic cancer WES data

In [1]:
# Download the dataset from the training-data folder
pancreatic_data = pd.read_csv('Pancreatic_cancer_tumor-normal.csv',  header=0, index_col=0)

NameError: name 'pd' is not defined

# Feature Selection with XGBoost

In [ ]:
#Create Column Target matching Dictionary value
map_labelMut = {'somatic_mutation': 1,
                'germline': 0}

pancreatic_data['target'] = pancreatic_data['label_mut'].map(map_labelMut)

In [ ]:
X_copy = pancreatic_data[['ExAC_ALL','SIFT_score','Polyphen2_HDIV_score','Polyphen2_HVAR_score',
                          'LRT_score','MutationTaster_score','MutationAssessor_score','FATHMM_score',
                          'PROVEAN_score','VEST3_score','MetaSVM_score','MetaLR_score','M-CAP_score',
                          'CADD_raw','CADD_phred','DANN_score','fathmm-MKL_coding_score','Eigen-raw',
                          'Eigen-PC-raw','GenoCanyon_score','integrated_fitCons_score','GERP++_RS',
                          'phyloP100way_vertebrate','phyloP20way_mammalian','phastCons100way_vertebrate',
                          'phastCons20way_mammalian','SiPhy_29way_logOdds','gnomAD_exome_ALL','FS',
                          'MQRankSum','QD','ReadPosRankSum','RPB','VAF','MQ','target']].copy()

In [ ]:
label = X_copy.columns[-1]
X = X_copy.drop(label, axis = 1)
Y = X_copy.drop(X, axis = 1)

## Split the train and test data

In [ ]:
seed = 42
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=seed)

## Imbalanced Dataset Handling
Because we have imbalanced datasets for the two classes, where the samples for "Confirmed Somatic Variant" dataset is much higher compare to the "Germline" class, we will create synthetic samples for the minority ("Germline") class. We used SMOTEENN for over-sampling (SMOTE) and noise cleaning (ENN)

### Define the parameters for SMOTE, ENN, and SMOTE-ENN

In [ ]:
## Create a SMOTE for oversampling
smote = SMOTE(sampling_strategy=0.2, k_neighbors=7)

# Create a ENN for undersampling
enn = EditedNearestNeighbours(sampling_strategy='majority', n_neighbors=10)

#Define the SMOTEENN resampling
smote_enn = SMOTEENN(enn=enn, smote=smote)

## Sampling the training data

In [ ]:
smote_enn = smote_enn.fit(x_train, y_train)
X_train_rsam, Y_train_rsam = smote_enn.fit_resample(x_train, y_train)

In [ ]:
Y_ohe_train= pd.get_dummies(Y_train_rsam).astype(float).values
Y_ohe_train= Y_train_rsam.values.ravel()
X_ohe_train = X_train_rsam.astype(float).values

## Sampling the testing data

In [ ]:
smote_enn = smote_enn.fit(x_test, y_test)
X_test_rsam, Y_test_rsam = smote_enn.fit_resample(x_test, y_test)

In [ ]:
Y_ohe_test = pd.get_dummies(Y_test_rsam).astype(float).values
Y_ohe_test = Y_test_rsam.values.ravel()
X_ohe_test = X_test_rsam.astype(float).values

## Loading data into DMatrices
In order to use the native API for XGBoost, we will first need to build DMatrices.

In [ ]:
dtrain = xgb.DMatrix(X_train_rsam, label=Y_ohe_train)
dtest = xgb.DMatrix(X_test_rsam, label=Y_ohe_test)

We are going to use mean absolute error (MAE) to evaluate the quality of our data train.

#### Train the model

In [ ]:
params = {
    'max_depth':3,
    'min_child_weight': 1,
    'eta': 0.1,
    'subsample': 1.0,
    'colsample_bytree': 0.1,
    'objective':'binary:logistic',
    'eval_metric': 'mae'
}

num_boost_round = 999

model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

In [ ]:
num_boost_round = model.best_iteration + 1
best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")]
)

Plot the features based on the feature importance score, in descending order

In [ ]:
fig1 = plt.gcf()
fig, ax = plt.subplots(figsize=(30,30))
xgb.plot_importance(loaded_model, max_num_features=100, height=0.5, ax=ax, importance_type='gain')
plt.show()

### Get the feature importance score according to 'gain' value

In [ ]:
feature_important = best_model.get_score(importance_type='gain')
keys = list(feature_important.keys())
values = list(feature_important.values())

The dataframe data_xgb below is consisted of the features ranked according to its feature importance score

In [ ]:
# Create the dataframe
data_xgb = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score", ascending=False)
print(data_xgb)

### Select the 24 features

In [ ]:
feature_24=data_xgb[:24]
print(feature_24.index)

# Somatic Mutation Classification with DNN

Select the 24 features from the pancreatic tumor-normal data as the input for the DNN model

In [ ]:
feature_24=pancreatic_data[['ExAC_ALL', 'FS', 'MetaLR_score', 'DANN_score', 'gnomAD_exome_ALL',
       'MutationTaster_score', 'GenoCanyon_score',
       'phastCons100way_vertebrate', 'MetaSVM_score', 'SIFT_score',
       'phyloP20way_mammalian', 'SiPhy_29way_logOdds', 'VEST3_score',
       'CADD_raw', 'Eigen-raw', 'M-CAP_score', 'MutationAssessor_score',
       'phyloP100way_vertebrate', 'QD', 'PROVEAN_score', 'RPB', 'MQ',
       'LRT_score', 'CADD_phred', 'label_mut']].copy()

## Separate features and labels

For the dataset with the full features:

In [ ]:
#label = pancreatic_data.columns[-1]
#X = pancreatic_data.drop(label, axis = 1)
#Y = pancreatic_data.drop(X, axis = 1)

For the dataset with the 24 features:

In [ ]:
label = feature_24.columns[-1]
X = feature_24.drop(label, axis = 1)
Y = feature_24.drop(X, axis = 1)

One-hot encode the labels

In [ ]:
print('One-hot encoding labels.')
Y_ohe= pd.get_dummies(Y).astype(float).values
print(Y_ohe)
# Get training data as numpy array
# because of non overlap
X_ohe = X.astype(float).values
print('Training shape is: ', X_ohe.shape)
print('Labels encoded shape is: ', Y_ohe.shape)

Split the train and test data for classification

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_ohe, Y_ohe, test_size=0.2, random_state=seed)

Fit the SMOTE-ENN to the training data of DNN

In [ ]:
smote_enn = smote_enn.fit(x_train, y_train)

## DNN Model Configuration

Define the F1-metric

In [ ]:
import tensorflow.keras.backend as K

def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

### Model configuration

In [ ]:
# Define the hyperparameters
batch_size=100
loss_function = categorical_crossentropy
num_classes = 2
no_epochs=100
optimizer = Adam(learning_rate=0.01)
verbosity = 1
num_folds = 10
input_shape = x_train.shape[1]

# Define the performance metrics for the model
METRICS = ['accuracy',
           tf.keras.metrics.AUC(name='auc'),
           tf.keras.metrics.Precision(name='precision'),
           tf.keras.metrics.Recall(name='recall'),
           tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
           tf.keras.metrics.TruePositives(name='tp'),
           tf.keras.metrics.FalsePositives(name='fp'),
           tf.keras.metrics.TrueNegatives(name='tn'),
           tf.keras.metrics.FalseNegatives(name='fn'),
           f1_metric
          ]

# Create Model
def make_model(metrics=METRICS, output_bias=None):
    model = Sequential([
        layers.Dense(64, input_dim=input_shape, activation='relu', kernel_initializer='normal', kernel_regularizer=l2(0.01), kernel_constraint=MaxNorm(5)),
        layers.Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
        layers.Dense(num_classes, kernel_initializer='normal', activation='softmax')
    ])
    
    # Compile model
    model.compile(
        loss=loss_function,
        optimizer=optimizer,
        metrics=METRICS)
    
    return model

Define per-fold training score containers

In [ ]:
tp_training_fold = []
fp_training_fold = []
tn_training_fold = []
fn_training_fold = []
loss_training_fold = []
auc_training_fold = []
acc_training_fold = []
precision_training_fold = []
recall_training_fold = []
training_f1_scores = []

Define per-fold validation score containers

In [ ]:
tp_per_fold = []
fp_per_fold = []
tn_per_fold = []
fn_per_fold = []
acc_per_fold = []
loss_per_fold = []
auc_per_fold = []
precision_per_fold = []
prc_per_fold = []
recall_per_fold = []
val_f1scores = []

## Create the DNN model

In [ ]:
model = make_model()

## Start the training

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
start = time.time()
for train, test in kfold.split(x_train, y_train):
    
    X_train_rsam, Y_train_rsam = smote_enn.fit_resample(x_train[train], y_train[train])
    Y_train_rsam = np.eye(2)[Y_train_rsam.flatten()]
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    # Fit data to model
    name_weights = "final_model_fold" + str(fold_no) + "_weights.h5"
    callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)
    history = model.fit(X_train_rsam, Y_train_rsam,
                        batch_size=batch_size,
                        epochs=no_epochs,
                        verbose=verbosity,
                        validation_data = (x_train[test], y_train[test]),
                        callbacks=callbacks)
    end = time.time()
    
    loss_training_fold.append(history.history['loss'])
    auc_training_fold.append(history.history['auc'] * 100)
    tp_training_fold.append(history.history['tp'])
    fp_training_fold.append(history.history['fp'])
    tn_training_fold.append(history.history['tn'])
    fn_training_fold.append(history.history['fn'])
    acc_training_fold.append(history.history['accuracy'] * 100)
    precision_training_fold.append(history.history['precision'] * 100)
    recall_training_fold.append(history.history['recall'] * 100)
    training_f1_scores.append(history.history['f1_metric'] * 100 )
    
    # Generate generalization metrics
    scores = model.evaluate(x_train[test], y_train[test], verbose=0)
    print(f'Validation score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%; {model.metrics_names[2]} of {scores[2]*100}%; {model.metrics_names[3]} of {scores[3] * 100}%; {model.metrics_names[4]} of {scores[4] * 100}%; {model.metrics_names[10]} of {scores[10] * 100}%')
    loss_per_fold.append(scores[0])
    acc_per_fold.append((scores[1] * 100))
    auc_per_fold.append((scores[2] * 100))
    precision_per_fold.append((scores[3] * 100))
    recall_per_fold.append((scores[4] * 100))
    prc_per_fold.append(scores[5])
    tp_per_fold.append(scores[6])
    fp_per_fold.append(scores[7])
    tn_per_fold.append(scores[8])
    fn_per_fold.append(scores[9])
    val_f1scores.append(round((scores[10] * 100), 2))
    # Increase fold number
    fold_no = fold_no + 1
    

# == Display average scores ==
print('------------------------------------------------------------------------')
print('Validation score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {(loss_per_fold[i])} - Accuracy: {acc_per_fold[i]}% - AUC: {auc_per_fold[i]}% - Precision: {precision_per_fold[i]}% - Recall: {recall_per_fold[i]}% - F1-score: {val_f1scores[i]}%')
print("Execution time for training is: %f"%(float(end)- float(start)))
print('------------------------------------------------------------------------')
print('Average training scores for all folds:')
print(f'> Loss: {np.mean(loss_training_fold)}')
print(f'> Accuracy: {(np.mean(acc_training_fold )* 100)} (+- {np.std(acc_training_fold)})')
print(f'> AUC: {(np.mean(auc_training_fold)* 100)} (+- {np.std(auc_training_fold)})')
print(f'> Precision: {(np.mean(precision_training_fold) * 100)} (+- {np.std(precision_training_fold)})')
print(f'> Recall: {(np.mean(recall_training_fold) * 100)} (+- {np.std(recall_training_fold)})')
print(f'> F1-score: {(np.mean(training_f1_scores) * 100)} (+- {np.std(training_f1_scores)})')
print('------------------------------------------------------------------------')
print('Average validation scores for all folds:')
print(f'> Loss: {np.mean(loss_per_fold)}')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> AUC: {np.mean(auc_per_fold)} (+- {np.std(auc_per_fold)})')
print(f'> Precision: {np.mean(precision_per_fold)} (+- {np.std(precision_per_fold)})')
print(f'> Recall: {np.mean(recall_per_fold)} (+- {np.std(recall_per_fold)})')
print(f'> F1-score: {np.mean(val_f1scores)} (+- {np.std(val_f1scores)})')
print('------------------------------------------------------------------------')

# Serialize Model to JSON
json_model = model.to_json()
with open('dnn_boost_pancreatic_24features.json', 'w') as json_file:
    json_file.write(json_model)
    
# Serialize weights to HDF5
model.save_weights('dnn_boost_pancreatic_24features')
print("Saved model to disk")
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print('Succesfully trained model!')

## Prediction of the testing data with the trained model

In [ ]:
# resample the testing data
smote_enn=smote_enn.fit(x_test, y_test)
x_test_resamp, y_test_resamp = smote_enn.fit_resample(x_test, y_test)
y_test_resamp = np.eye(2)[y_test_resamp.flatten()]

Start the prediction of the testing data

In [ ]:
y_test_pred_cat = model.predict(x_test_resamp, batch_size=100, verbose=verbosity).round()
accuracy_final = accuracy_score(y_test_resamp, y_test_pred_cat)
recall_final = recall_score(y_test_resamp, y_test_pred_cat, average='weighted')
precision_final = precision_score(y_test_resamp, y_test_pred_cat, average='weighted')
#calculate F1 score
f1_final = f1_score(y_test_resamp, y_test_pred_cat, average='weighted')
print('Accuracy of prediction model:')
print(f'> Prediction accuracy: {(accuracy_final * 100)} ')
print('Precision of prediction model:')
print(f'> Precision: {(precision_final * 100)} ')
print('Recall of prediction model:')
print(f'> Recall: {(recall_final * 100)} ')
print('F1-score of prediction model:')
print(f'> F1: {(f1_final * 100)} ')

### Plot the Confusion Matrix

In [ ]:
def plot_cm(labels, predictions, p=0.5):
    cm = confusion_matrix(test_true_label, predicted_label)
    plt.figure(figsize=(5,5))
    sns.heatmap(cm, annot=True, fmt="d")
    plt.title('Confusion matrix')
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    
    print('True Somatic Variant Detected (True Positives): ', cm[1][1])
    print('Germline Variant Detected (True Negatives): ', cm[0][0])
    print('Germline Variant Incorrectly Detected as True Somatic Variant (False Positives): ', cm[0][1])
    print('True Somatic Variant Detected Incorrectly Detected as Germline (False Negatives): ', cm[1][0])
    print('Total True Somatic Variant: ', np.sum(cm[1]))
    print('Total Germline: ', np.sum(cm[0]))

In [ ]:
test_true_label = tf.argmax(y_test_resamp, axis = 1)
predicted_label = tf.argmax(y_test_pred_cat, axis = 1)
plot_cm(test_true_label, predicted_label)

# Classification of the tumor-only dataset

In [ ]:
# Download the dataset from the tumor-only_data folder
tumor_only_data = pd.read_csv('Pancreatic_cancer_tumor-only.csv',  header=0, index_col=0)

## Select the same 24 features according to the XGBoost feature importance test from the tumor-normal data

In [ ]:
tumor_only_feature24=tumor_only_data[['ExAC_ALL', 'FS', 'MetaLR_score', 'DANN_score', 'gnomAD_exome_ALL',
       'MutationTaster_score', 'GenoCanyon_score',
       'phastCons100way_vertebrate', 'MetaSVM_score', 'SIFT_score',
       'phyloP20way_mammalian', 'SiPhy_29way_logOdds', 'VEST3_score',
       'CADD_raw', 'Eigen-raw', 'M-CAP_score', 'MutationAssessor_score',
       'phyloP100way_vertebrate', 'QD', 'PROVEAN_score', 'RPB', 'MQ',
       'LRT_score', 'CADD_phred','label']].copy()

Create Column Target matching Dictionary value

In [ ]:
map_labelMut = {'somatic_mutation': 1,
                'germline polymorphism': 0}

tumor_only_feature24['target'] = tumor_only_feature24['label'].map(map_labelMut)

Separate the features and labels

In [ ]:
label = tumor_only_feature24.columns[-1]
X = tumor_only_feature24.drop(label, axis = 1)
Y = tumor_only_feature24.drop(X, axis = 1)

In [ ]:
X_ohe = X.astype(float).values
Y_ohe = Y['target'].values
Y_true = np.eye(2)[Y_ohe.flatten()]

## Predict the tumor-only dataset using the trained model from the classifier

### Pull in model from output folder

In [ ]:
json_file = open('dnn_boost_pancreatic_24features.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model_24 = model_from_json(loaded_model_json)

### Load weights into new model

In [ ]:
loaded_model_24.load_weights('dnn_boost_pancreatic_24features')
print("Loaded model from disk")
print()

### Classify the tumor-only dataset

In [ ]:
y_pred_24 = loaded_model_24.predict(X_ohe, batch_size=100, verbose=1).round()
accuracy_final = accuracy_score(Y_true, y_pred_24)
recall_final = recall_score(Y_true, y_pred_24, average='weighted')
precision_final = precision_score(Y_true, y_pred_24, average='weighted')
#calculate F1 score
f1_final = f1_score(Y_true, y_pred_24, average='weighted')
print('Accuracy of prediction model:')
print(f'> Prediction accuracy: {(accuracy_final * 100)} ')
print('Precision of prediction model:')
print(f'> Precision: {(precision_final * 100)} ')
print('Recall of prediction model:')
print(f'> Recall: {(recall_final * 100)} ')
print('F1-score of prediction model:')
print(f'> F1: {(f1_final * 100)} ')

### Plot the confusion matrix

In [ ]:
test_true_label = tf.argmax(Y_true, axis = 1)
predicted_label = tf.argmax(y_pred_24, axis = 1)
plot_cm(test_true_label, predicted_label)

# Summary

In this demo, we showed the workflow of DNN-Boost, consisted of feature selection with XGBoost and classification with four-layers DNN.